In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/winter-2025-machine-learning-competition-1-p/sample_submission.csv
/kaggle/input/winter-2025-machine-learning-competition-1-p/train.csv
/kaggle/input/winter-2025-machine-learning-competition-1-p/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/winter-2025-machine-learning-competition-1-p/train.csv')
test = pd.read_csv('/kaggle/input/winter-2025-machine-learning-competition-1-p/test.csv',index_col=[0])

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import roc_auc_score

In [4]:
train_df = train.copy()
y = train_df.pop("Label")
X = train_df.drop(['X3', 'X1'],axis=1)
test = test.drop(['X3', 'X1'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)

In [5]:
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

In [6]:
rf = RandomForestClassifier(random_state=0, n_jobs=-1)

In [7]:
rf_search = RandomizedSearchCV(rf, param_distributions=param_dist, 
                               n_iter=10, scoring='roc_auc', 
                               cv=3, verbose=2, random_state=42, n_jobs=-1)

In [8]:
rf_search.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3,
                   estimator=RandomForestClassifier(n_jobs=-1, random_state=0),
                   n_jobs=-1,
                   param_distributions={'max_depth': [10, 20],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2],
                                        'min_samples_split': [2, 5],
                                        'n_estimators': [100, 200, 300]},
                   random_state=42, scoring='roc_auc', verbose=2)

In [9]:
best_rf = rf_search.best_estimator_
best_rf

RandomForestClassifier(max_depth=10, max_features='log2', min_samples_leaf=2,
                       n_estimators=200, n_jobs=-1, random_state=0)

In [10]:
best_rf = RandomForestClassifier(
    n_estimators=rf_search.best_params_['n_estimators'],
    max_depth=rf_search.best_params_['max_depth'],
    min_samples_split=rf_search.best_params_['min_samples_split'],
    min_samples_leaf=rf_search.best_params_['min_samples_leaf'],
    max_features=rf_search.best_params_['max_features'],
    bootstrap=True,
    n_jobs=-1,
    random_state=0
)

In [11]:
best_rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, max_features='log2', min_samples_leaf=2,
                       n_estimators=200, n_jobs=-1, random_state=0)

In [12]:
pred_train = best_rf.predict_proba(X_train)[:, 1]
pred_test = best_rf.predict_proba(X_test)[:, 1]
roc_auc_score(y_train, pred_train, max_fpr=.01), roc_auc_score(y_test, pred_test, max_fpr=.01)

(0.9930672713489308, 0.8626483256381267)

In [13]:
best_rf.fit(X, y)

RandomForestClassifier(max_depth=10, max_features='log2', min_samples_leaf=2,
                       n_estimators=200, n_jobs=-1, random_state=0)

In [14]:
test_probs = best_rf.predict_proba(test)[:, 1]

In [15]:
submission = pd.DataFrame({'id':test.index, 'Label':test_probs})
submission.set_index('id').to_csv("submission.csv")